In [69]:
import os
import sys
import tarfile
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from six.moves import urllib
from tensorflow.python.framework import ops
import IPython
ops.reset_default_graph()

sess = tf.Session()

In [70]:
batch_size = 128
data_dir = 'temp'
output_every = 50
generations = 20000
eval_every = 500
image_height = 32
image_width = 32
crop_height = 24
crop_width = 24
num_channels = 3
num_targets = 10
extract_folder = 'cifar-10-batches-bin'

In [71]:
# 学習率を幾何級数的に減衰させるためのパラメータ
learning_rate = 0.1
lr_decay = 0.1
num_gens_to_wait = 250.

In [72]:
image_vec_length = image_height * image_width * num_channels
record_length = 1 + image_vec_length # 1はラベル（0-9）の分

In [73]:
# データをロードするためのディレクトリとURL
data_dir = 'temp'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
cifar10_url = 'http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'

# ファイルが存在しない場合はデータをダウンロード
data_file = os.path.join(data_dir, 'cifar-10-binary.tar.gz')
if os.path.isfile(data_file):
    pass
else:
    # ファイルをダウンロード
    def progress(block_num, block_size, total_size):
        progress_info = [cifar10_url, float(block_num * block_size) / float(total_size) * 100,0]
        print('\r Downloading {} - {:.2f}%'.format(*progress_info), end="")
    filepath, _ = urllib.request.urlretrieve(cifar10_url, data_file, progress)
    # ファイルを展開
    tarfile.open(filepath, 'r:gz').extractall(data_dir)

In [74]:
def read_cifar_files(filename_queue, distort_images = True):
    reader = tf.FixedLengthRecordReader(record_bytes=record_length)
    key, record_string = reader.read(filename_queue)
    record_bytes = tf.decode_raw(record_string, tf.uint8)
    
    # ラベルを抽出
    image_label = tf.cast(tf.slice(record_bytes, [0], [1]), tf.int32)
    
    # 画像を抽出
    image_extracted = tf.reshape(tf.slice(record_bytes, [1], [image_vec_length]),
                                [num_channels, image_height, image_width])
    
    # 画像を変形
    image_uint8image = tf.transpose(image_extracted, [1, 2, 0])
    reshaped_image = tf.cast(image_uint8image, tf.float32)
    
    # 画像をランダムに切り取る
    final_image = tf.image.resize_image_with_crop_or_pad(reshaped_image, crop_width, crop_height)
    
    if distort_images:
        # 画像の左右を反転させ、明るさとコントラストを変更
        final_image = tf.image.random_flip_left_right(final_image)
        final_image = tf.image.random_brightness(final_image,max_delta=63)
        final_image = tf.image.random_contrast(final_image,lower=0.2,upper=1.8)
        
    # 画像を正規化
    final_image = tf.image.per_image_standardization(final_image)
    return(final_image, image_label)

In [75]:
def input_pipeline(batch_size, train_logical=True):
    if train_logical:
        files = [os.path.join(data_dir, extract_folder, 'data_batch_{}.bin'.format(i)) for i in range(1,6)]
    else:
        files = [os.path.join(data_dir, extract_folder, 'test_batch.bin')]
    filename_queue = tf.train.string_input_producer(files)
    image, label = read_cifar_files(filename_queue)
    
    min_after_dequeue = 5000
    capacity = min_after_dequeue + 3 * batch_size
    example_batch, label_batch = tf.train.shuffle_batch([image, label],
                                                       batch_size=batch_size,
                                                       capacity=capacity,
                                                       min_after_dequeue=min_after_dequeue)
    
    return(example_batch, label_batch)

In [76]:
def cifar_cnn_model(input_images, batch_size, train_logical=True):
    def truncated_normal_var(name, shape, dtype):
        return(tf.get_variable(name=name, shape=shape, dtype=dtype, initializer=tf.truncated_normal_initializer(stddev=0.05)))
    def zero_var(name, shape, dtype):
        return(tf.get_variable(name=name, shape=shape, dtype=dtype, initializer=tf.constant_initializer(0.0)))
    
    # 1つ目の畳み込み層
    with tf.variable_scope('conv1') as scope:
        # 畳み込みカーネルは3色すべてで 5 x 5 、ここでは64個の特徴量を抽出
        conv1_kernel = truncated_normal_var(name='conv_kernel1', shape=[5, 5, 3, 64], dtype=tf.float32)
        # 画像全体をストライド1で畳み込む
        conv1 = tf.nn.conv2d(input_images, conv1_kernel, [1, 1, 1, 1], padding='SAME')
        # バイアス項を初期化した上で追加
        conv1_bias = zero_var(name='conv_bias1', shape=[64], dtype=tf.float32)
        conv1_add_bias = tf.nn.bias_add(conv1, conv1_bias)
        # ReLU活性化関数
        relu_conv1 = tf.nn.relu(conv1_add_bias)
        
    # マックスプーリング
    pool1 = tf.nn.max_pool(relu_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool_layer1')
    
    # LRN （Local Response Normalization）（以下の論文を参照）
    # 論文：http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks
    norm1 = tf.nn.lrn(pool1, depth_radius=5, bias=2.0, alpha=1e-3, beta=0.75, name='norm1')
    
    # 2つ目の畳み込み層
    with tf.variable_scope('conv2') as scope:
        # 畳み込みカーネルは1つ前の64個の特徴量すべてにわたって 5 x 5
        # さらに64個の特徴量を抽出
        conv2_kernel = truncated_normal_var(name='conv_kernel2', shape=[5, 5, 64, 64], dtype=tf.float32)
        # 1つ前の出力全体をストライド1で畳み込む
        conv2 = tf.nn.conv2d(norm1, conv2_kernel, [1, 1, 1, 1], padding='SAME')
        # バイアス項を初期化した上で追加
        conv2_bias = zero_var(name='conv_bias2', shape=[64], dtype=tf.float32)
        conv2_add_bias = tf.nn.bias_add(conv2, conv2_bias)
        # ReLU活性化関数
        relu_conv2 = tf.nn.relu(conv2_add_bias)
        
    # マックスプーリング
    pool2 = tf.nn.max_pool(relu_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool_layer2')
    
    # LRN（Local Response Normalization）（前の論文を参照）
    norm2 = tf.nn.lrn(pool2, depth_radius=5, bias=2.0, alpha=1e-3, beta=0.75, name='norm2')
    
    # 全結合祖のために出力を1つの行列にまとめて乗算できるようにする
    reshaped_output = tf.reshape(norm2, [batch_size, -1])
    reshaped_dim = reshaped_output.get_shape()[1].value
    
    # 1つ目の全結合層
    with tf.variable_scope('full1') as scope:
        # 1つ目の全結合層の出力は384層
        full_weight1 = truncated_normal_var(name='full_mult1', shape=[reshaped_dim, 384], dtype=tf.float32)
        full_bias1 = zero_var(name='full_bias1', shape=[384], dtype=tf.float32)
        full_layer1 = tf.nn.relu(tf.add(tf.matmul(reshaped_output, full_weight1), full_bias1))
        
    # 2つ目の全結合層
    with tf.variable_scope('full2') as scope:
        # 2つ目の全結合層の出力は192個
        full_weight2 = truncated_normal_var(name='full_mult2', shape=[384, 192], dtype=tf.float32)
        full_bias2 = zero_var(name='full_bias2', shape=[192], dtype=tf.float32)
        full_layer2 = tf.nn.relu(tf.add(tf.matmul(full_layer1, full_weight2), full_bias2))
        
    # 最後の全結合層 -> 出力は10個のカテゴリ
    with tf.variable_scope('full3') as scope:
        # 最後の全結合層の出力は10個（num_targets）
        full_weight3 = truncated_normal_var(name='full_mult3', shape=[192, num_targets], dtype=tf.float32)
        full_bias3 = zero_var(name='full_bias3', shape=[num_targets], dtype=tf.float32)
        final_output = tf.add(tf.matmul(full_layer2, full_weight3), full_bias3)
        
    return(final_output)

In [77]:
def cifar_loss(logits, targets):
    # 余分な次元を削除し、目的値を整数にキャスト
    targets = tf.squeeze(tf.cast(targets, tf.int32))
    # ロジットと目的値
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=targets)
    # バッチサイズ全体の損失値を計算
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    return(cross_entropy_mean)

In [78]:
def train_step(loss_value, generation_num):
    # 学習率を幾何級数的に減衰（引き下げ）
    model_learning_rate = tf.train.exponential_decay(learning_rate, generation_num,
                                                    num_gens_to_wait, lr_decay, staircase=True)
    # 最適化関数を作成
    my_optimizer = tf.train.GradientDescentOptimizer(model_learning_rate)
    # トレーニングステップを初期化
    train_step = my_optimizer.minimize(loss_value)
    return(train_step)

In [79]:
def accuracy_of_batch(logits, targets):
    # 目的値が整数であることを確実にし、余分な次元を削除
    targets = tf.squeeze(tf.cast(targets, tf.int32))
    # ロジットが最大になるものを探すことで、予測値を取得
    batch_predictions = tf.cast(tf.argmax(logits, 1), tf.int32)
    # それらがバッチ全体で等しいかどうかを確認
    predicted_correctly = tf.equal(batch_predictions, targets)
    # バッチ全体で1（True）と0（False）の平均を求める
    accuracy = tf.reduce_mean(tf.cast(predicted_correctly, tf.float32))
    return(accuracy)

In [80]:
# データパイプラインを初期化
images, targets = input_pipeline(batch_size, train_logical=True)
# パイプラインからテスト用のバッチ画像と目的値を取得
test_images, test_targets = input_pipeline(batch_size, train_logical=False)

In [81]:
# モデルを設定
with tf.variable_scope('model_definition') as scope:
    # トレーニングモデルを設定
    model_output = cifar_cnn_model(images, batch_size)
    # 変数を再利用するためにスコープを設定
    scope.reuse_variables()
    # テストモデルの出力を設定
    test_output = cifar_cnn_model(test_images, batch_size)

In [82]:
# 損失関数を設定
loss = cifar_loss(model_output, targets)

# 正解関数を設定
accuracy = accuracy_of_batch(test_output, test_targets)

# トレーニング演算を作成
generation_num = tf.Variable(0, trainable=False)
train_op = train_step(loss, generation_num)

In [83]:
# 変数を初期化
init = tf.global_variables_initializer()
sess.run(init)

# キューを初期化
tf.train.start_queue_runners(sess=sess)

[<Thread(Thread-3, started daemon 139939884824320)>,
 <Thread(Thread-4, started daemon 139939876431616)>,
 <Thread(Thread-5, started daemon 139939868038912)>,
 <Thread(Thread-6, started daemon 139939859646208)>]

In [ ]:
train_loss = []
test_accuracy = []
for i in range(generations):
    _, loss_value = sess.run([train_op, loss])
    
    if (i+1) % output_every == 0:
        train_loss.append(loss_value)
        output = 'Generation {}: Loss = {:.5f}'.format((i+1), loss_value)
        print(output)
        
    if (i+1) % eval_every == 0:
        [temp_accuracy] = sess.run([accuracy])
        test_accuracy.append(temp_accuracy)
        acc_output = ' --- Test Accuracy = {:.2f}%.'.format(100.*temp_accuracy)

Generation 50: Loss = 1.97588
Generation 100: Loss = 1.89741
Generation 150: Loss = 1.90721
Generation 200: Loss = 1.67443
Generation 250: Loss = 1.59165
Generation 300: Loss = 1.64054
Generation 350: Loss = 1.55082
Generation 400: Loss = 1.47431
Generation 450: Loss = 1.41637
Generation 500: Loss = 1.33618
Generation 550: Loss = 1.32995
Generation 600: Loss = 1.35040
Generation 650: Loss = 1.15915
Generation 700: Loss = 1.25833
Generation 750: Loss = 1.25837
Generation 800: Loss = 1.13081
Generation 850: Loss = 1.14150
Generation 900: Loss = 1.37401
Generation 950: Loss = 1.26266
Generation 1000: Loss = 1.26377
Generation 1050: Loss = 1.12136
Generation 1100: Loss = 1.19865
Generation 1150: Loss = 1.05593
Generation 1200: Loss = 1.04289
Generation 1250: Loss = 1.22123
Generation 1300: Loss = 1.11560
Generation 1350: Loss = 0.95127
Generation 1400: Loss = 1.13910
Generation 1450: Loss = 1.10516
Generation 1500: Loss = 1.16113
Generation 1550: Loss = 1.11844
Generation 1600: Loss = 0.99

In [ ]:
eval_indices = range(0, generations, eval_every)
output_indices = range(0, generations, output_every)

# 損失値をプロット
plt.plot(output_indices, train_loss, 'k-')
plt.title('Softmax Loss per Generation')
plt.xlabel('Generation')
plt.ylabe;('Softmax Loss')
plt.show()

# 正解率をプロット
plt.plot(eval_indices, test_accuracy, 'k-')
plt.title('Test Accuracy')
plt.xlabel('Generation')
plt.ylabel('Accuracy')
plt.show()